# LLM Optimization Modelling Experiment

In [156]:
import vertexai
from vertexai.preview.generative_models import GenerativeModel
from IPython.display import Markdown

## 1. Define the problem description

In [466]:
problem = '''The PRODA, S.A. industrial products firm has to face the problem of scheduling
the weekly production of its three products (P1, P2 and P3). These products are
sold to large industrial firms and PRODA, S.A. wishes to supply its products in
quantities that are more profitable for it.

Each product entails three operations contributing to the costs: smelting; mechanisation; assembly and
packaging. The smelting operations for products P1 and P2 could be subcontracted, but the smelting operation for product P3 requires special equipment, thus
preventing the use of subcontracts. PRODA also want to know, how much they should subcontract.

For product P1 the direct unit costs of all possible operations are:
- smelting at PRODA: 0.30$
- subcontracted smelting: 0.50$
- mechanisation: 0.20$
- Assembly and packaging: 0.3$
The unit sales price is 1.50$.

For product P2 the direct unit costs of all possible operations are:
- smelting at PRODA: 0.50$
- subcontracted smelting: 0.60$
- mechanisation: 0.10$
- Assembly and packaging: 0.20$
The unit sales price is 1.80$.

For product P3 the direct unit costs of all possible operations are:
- smelting at PRODA: 0.40$
- mechanisation: 0.27$
- Assembly and packaging: 0.20$
The unit sales price is 1.97$.

Each unit of product P1 requires 6 min of smelting time (if performed at PRODA, S.A.), 6 min of mechanisation time and 3 min of assembly and packaging time, respectively. For product P2, the times are 10, 3 and 2 min, respectively. One unit of product P3 needs 8 min of smelting time, 8 min of mechanisation and 2 min for assembly and packaging. PRODA, S.A. has weekly capacities of 8,000 min of smelting time, 12,000 min of mechanisation time and 10,000 min of assembly and packaging time.
The objective is to maximize weekly profits.'''

## 2. Generate the mathematical model

In [467]:
#Initializing the session. To replicate, make sure the right credentials are saved in a PATH variable
PROJECT_ID = "llm4optproblems"
REGION = "us-central1"
vertexai.init(project=PROJECT_ID, location=REGION)

#Specifying the model
generative_multimodal_model = GenerativeModel("gemini-1.5-pro-preview-0409")

#The propmt applied to all problems
prompt = '''Let's think step by step. Please write a mathematical optimization model for this problem. If there are parameter values, make sure to include them in the mathematical formulation.
'''

#Generate the response
response = generative_multimodal_model.generate_content([prompt+problem])


In [468]:
#Show the resopnse in a formatted way
Markdown(response.text)

## Mathematical Optimization Model for PRODA, S.A. Weekly Production

**Sets:**

* $P = \{P1, P2, P3\}$: Set of products.

**Parameters:**

* **Production Costs:**
    * $c_{i}^{s}$: Unit cost of smelting product $i$ at PRODA, $\forall i \in P$.
    * $c_{i}^{ss}$: Unit cost of subcontracted smelting for product $i$, $\forall i \in \{P1, P2\}$.
    * $c_{i}^{m}$: Unit cost of mechanisation for product $i$, $\forall i \in P$.
    * $c_{i}^{ap}$: Unit cost of assembly and packaging for product $i$, $\forall i \in P$.
* **Sales Price:**
    * $p_{i}$: Unit sales price for product $i$, $\forall i \in P$.
* **Production Times:**
    * $t_{i}^{s}$: Time (in minutes) required for smelting one unit of product $i$ at PRODA, $\forall i \in P$.
    * $t_{i}^{m}$: Time (in minutes) required for mechanization of one unit of product $i$, $\forall i \in P$.
    * $t_{i}^{ap}$: Time (in minutes) required for assembly and packaging of one unit of product $i$, $\forall i \in P$.
* **Capacity:**
    * $C^{s}$: Total available smelting time at PRODA per week (in minutes).
    * $C^{m}$: Total available mechanization time per week (in minutes).
    * $C^{ap}$: Total available assembly and packaging time per week (in minutes).

**Parameter Values:**

| Parameter | P1  | P2  | P3  |
| :-------- | :---- | :---- | :---- |
| $c_{i}^{s}$ | 0.30 | 0.50 | 0.40 |
| $c_{i}^{ss}$ | 0.50 | 0.60 | -    |
| $c_{i}^{m}$ | 0.20 | 0.10 | 0.27 |
| $c_{i}^{ap}$ | 0.30 | 0.20 | 0.20 |
| $p_{i}$ | 1.50 | 1.80 | 1.97 |
| $t_{i}^{s}$ | 6    | 10   | 8    |
| $t_{i}^{m}$ | 6    | 3    | 8    |
| $t_{i}^{ap}$ | 3    | 2    | 2    |

* $C^{s} = 8000$ minutes
* $C^{m} = 12000$ minutes
* $C^{ap} = 10000$ minutes

**Decision Variables:**

* $x_{i}$: Number of units of product $i$ produced at PRODA, $\forall i \in P$.
* $y_{i}$: Number of units of product $i$ subcontracted for smelting, $\forall i \in \{P1, P2\}$.

**Objective Function:**

Maximize weekly profit:

$$
\sum_{i \in P} p_{i} (x_{i} + y_{i}) - \sum_{i \in P} ( c_{i}^{s} x_{i} + c_{i}^{m} x_{i} + c_{i}^{ap} x_{i}) - \sum_{i \in \{P1, P2\}} c_{i}^{ss} y_{i}
$$

**Constraints:**

* **Smelting Capacity:**
$$
\sum_{i \in P} t_{i}^{s} x_{i} \leq C^{s}
$$
* **Mechanization Capacity:**
$$
\sum_{i \in P} t_{i}^{m} x_{i} \leq C^{m}
$$
* **Assembly and Packaging Capacity:**
$$
\sum_{i \in P} t_{i}^{ap} x_{i} \leq C^{ap}
$$
* **Non-negativity:**
$$
x_{i} \geq 0, \forall i \in P
$$
$$
y_{i} \geq 0, \forall i \in \{P1, P2\} 
$$

This model will determine the optimal production quantities for each product at PRODA and through subcontracting, maximizing the weekly profits while adhering to the production capacity constraints. 


## 3. Generate the pyomo code

In [469]:
#Second prompt gets the output of the previous step and generates the code
prompt2 = "Please write pyomo code for this mathematical problem. Use sample data where needed. Indicate where you use sample data. \n"
prompt2 += response.text 
response2 = generative_multimodal_model.generate_content([prompt2])

In [470]:
#Showing the code in a formatted way
Markdown(response2.text)

```python
import pyomo.environ as pyo

# Define sets
P = ['P1', 'P2', 'P3']

# Define parameters (Sample Data)
c_s = {'P1': 0.30, 'P2': 0.50, 'P3': 0.40}
c_ss = {'P1': 0.50, 'P2': 0.60}
c_m = {'P1': 0.20, 'P2': 0.10, 'P3': 0.27}
c_ap = {'P1': 0.30, 'P2': 0.20, 'P3': 0.20}
p = {'P1': 1.50, 'P2': 1.80, 'P3': 1.97}
t_s = {'P1': 6, 'P2': 10, 'P3': 8}
t_m = {'P1': 6, 'P2': 3, 'P3': 8}
t_ap = {'P1': 3, 'P2': 2, 'P3': 2}
C_s = 8000
C_m = 12000
C_ap = 10000

# Create model
model = pyo.ConcreteModel()

# Define variables
model.x = pyo.Var(P, nonneg=True) # units produced at PRODA
model.y = pyo.Var(['P1', 'P2'], nonneg=True) # units subcontracted

# Define objective function
model.profit = pyo.Objective(
    expr = sum(p[i] * (model.x[i] + model.y[i]) for i in P) - 
           sum((c_s[i] + c_m[i] + c_ap[i]) * model.x[i] for i in P) -
           sum(c_ss[i] * model.y[i] for i in ['P1', 'P2']),
    sense=pyo.maximize
)

# Define constraints
model.smelting_capacity = pyo.Constraint(expr = sum(t_s[i] * model.x[i] for i in P) <= C_s)
model.mechanization_capacity = pyo.Constraint(expr = sum(t_m[i] * model.x[i] for i in P) <= C_m)
model.assembly_capacity = pyo.Constraint(expr = sum(t_ap[i] * model.x[i] for i in P) <= C_ap)

# Solve the model
solver = pyo.SolverFactory('glpk')  # You can choose a different solver if you prefer
solver.solve(model)

# Print results
print("Optimal Production Plan:")
for i in P:
    print(f"Product {i}: Produce {model.x[i]()} at PRODA, Subcontract {model.y[i]() if i in ['P1', 'P2'] else 0}")
print(f"Total Profit: {model.profit()}") 
```

This code first defines the sets and parameters based on the sample data provided. Then, a Pyomo model is created with variables for production quantities (`x`) and subcontracted quantities (`y`). The objective function is defined to maximize profit, and constraints are added for smelting, mechanization, and assembly capacities. Finally, the model is solved using the GLPK solver, and the optimal production plan and total profit are printed. 


## 4. Input problem data and try running the generated code

In [471]:
import pyomo.environ as pyo

# Define sets
P = ['P1', 'P2', 'P3']

# Define parameters (Sample Data)
c_s = {'P1': 0.30, 'P2': 0.50, 'P3': 0.40}
c_ss = {'P1': 0.50, 'P2': 0.60}
c_m = {'P1': 0.20, 'P2': 0.10, 'P3': 0.27}
c_ap = {'P1': 0.30, 'P2': 0.20, 'P3': 0.20}
p = {'P1': 1.50, 'P2': 1.80, 'P3': 1.97}
t_s = {'P1': 6, 'P2': 10, 'P3': 8}
t_m = {'P1': 6, 'P2': 3, 'P3': 8}
t_ap = {'P1': 3, 'P2': 2, 'P3': 2}
C_s = 8000
C_m = 12000
C_ap = 10000

# Create model
model = pyo.ConcreteModel()

# Define variables
model.x = pyo.Var(P, nonneg=True) # units produced at PRODA
model.y = pyo.Var(['P1', 'P2'], nonneg=True) # units subcontracted

# Define objective function
model.profit = pyo.Objective(
    expr = sum(p[i] * (model.x[i] + model.y[i]) for i in P) - 
           sum((c_s[i] + c_m[i] + c_ap[i]) * model.x[i] for i in P) -
           sum(c_ss[i] * model.y[i] for i in ['P1', 'P2']),
    sense=pyo.maximize
)

# Define constraints
model.smelting_capacity = pyo.Constraint(expr = sum(t_s[i] * model.x[i] for i in P) <= C_s)
model.mechanization_capacity = pyo.Constraint(expr = sum(t_m[i] * model.x[i] for i in P) <= C_m)
model.assembly_capacity = pyo.Constraint(expr = sum(t_ap[i] * model.x[i] for i in P) <= C_ap)

# Solve the model
solver = pyo.SolverFactory('glpk')  # You can choose a different solver if you prefer
solver.solve(model)

# Print results
print("Optimal Production Plan:")
for i in P:
    print(f"Product {i}: Produce {model.x[i]()} at PRODA, Subcontract {model.y[i]() if i in ['P1', 'P2'] else 0}")
print(f"Total Profit: {model.profit()}") 

ValueError: Unexpected keyword options found while constructing 'IndexedVar':
	nonneg

## 5. Correct the code to verify model viability (optional)

## 6. Printing the outputs as strings, so they can be saved.
Those can be rendered as markdown for better readability

In [474]:
print(response.text)

## Mathematical Optimization Model for PRODA, S.A. Weekly Production

**Sets:**

* $P = \{P1, P2, P3\}$: Set of products.

**Parameters:**

* **Production Costs:**
    * $c_{i}^{s}$: Unit cost of smelting product $i$ at PRODA, $\forall i \in P$.
    * $c_{i}^{ss}$: Unit cost of subcontracted smelting for product $i$, $\forall i \in \{P1, P2\}$.
    * $c_{i}^{m}$: Unit cost of mechanisation for product $i$, $\forall i \in P$.
    * $c_{i}^{ap}$: Unit cost of assembly and packaging for product $i$, $\forall i \in P$.
* **Sales Price:**
    * $p_{i}$: Unit sales price for product $i$, $\forall i \in P$.
* **Production Times:**
    * $t_{i}^{s}$: Time (in minutes) required for smelting one unit of product $i$ at PRODA, $\forall i \in P$.
    * $t_{i}^{m}$: Time (in minutes) required for mechanization of one unit of product $i$, $\forall i \in P$.
    * $t_{i}^{ap}$: Time (in minutes) required for assembly and packaging of one unit of product $i$, $\forall i \in P$.
* **Capacity:**
    * $C

In [475]:
print(response2.text)

```python
import pyomo.environ as pyo

# Define sets
P = ['P1', 'P2', 'P3']

# Define parameters (Sample Data)
c_s = {'P1': 0.30, 'P2': 0.50, 'P3': 0.40}
c_ss = {'P1': 0.50, 'P2': 0.60}
c_m = {'P1': 0.20, 'P2': 0.10, 'P3': 0.27}
c_ap = {'P1': 0.30, 'P2': 0.20, 'P3': 0.20}
p = {'P1': 1.50, 'P2': 1.80, 'P3': 1.97}
t_s = {'P1': 6, 'P2': 10, 'P3': 8}
t_m = {'P1': 6, 'P2': 3, 'P3': 8}
t_ap = {'P1': 3, 'P2': 2, 'P3': 2}
C_s = 8000
C_m = 12000
C_ap = 10000

# Create model
model = pyo.ConcreteModel()

# Define variables
model.x = pyo.Var(P, nonneg=True) # units produced at PRODA
model.y = pyo.Var(['P1', 'P2'], nonneg=True) # units subcontracted

# Define objective function
model.profit = pyo.Objective(
    expr = sum(p[i] * (model.x[i] + model.y[i]) for i in P) - 
           sum((c_s[i] + c_m[i] + c_ap[i]) * model.x[i] for i in P) -
           sum(c_ss[i] * model.y[i] for i in ['P1', 'P2']),
    sense=pyo.maximize
)

# Define constraints
model.smelting_capacity = pyo.Constraint(expr = sum(t_s[